In [1]:
# http://blog.theeluwin.kr/post/147587579528/python-crfsuite%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%B4%EC%84%9C-%ED%95%9C%EA%B5%AD%EC%96%B4-%EC%9E%90%EB%8F%99-%EB%9D%84%EC%96%B4%EC%93%B0%EA%B8%B0%EB%A5%BC-%ED%95%99%EC%8A%B5%ED%95%B4%EB%B3%B4%EC%9E%90

In [26]:
import os
import re
import codecs

In [ ]:
import pycrfsuite

In [5]:
from deep_learning.config import INPUT_DIR, MYSQL

In [17]:
def raw2corpus(raw_path, corpus_path):
    """
    raw -> 학습용 데이터포맷으로 바꿔줌
    """
    raw = codecs.open(raw_path, encoding='utf-8')
    raw_sentences = raw.read().split('\n')
    corpus = codecs.open(corpus_path, 'w', encoding='utf-8')
    sentences = []
    for raw_sentence in raw_sentences:
        if not raw_sentence:
            continue
        text = re.sub(r'(\ )+', ' ', raw_sentence).strip()
        taggeds = []
        for i in range(len(text)):
            if i == 0:
                taggeds.append('{}/B'.format(text[i]))
            elif text[i] != ' ':
                successor = text[i - 1]
                if successor == ' ':
                    taggeds.append('{}/B'.format(text[i]))
                else:
                    taggeds.append('{}/I'.format(text[i]))
        sentences.append(' '.join(taggeds))
    corpus.write('\n'.join(sentences))

In [18]:
raw2corpus(os.path.join(INPUT_DIR, 'doc1.0e+4_typetitle.sentences'), 'train.txt')

In [19]:
!head train.txt

한/B 미/I f/B t/I a/I 이/B 행/I 위/I ,/I 美/B 관/B 세/I 폭/I 탄/I 근/B 거/I 규/B 정/I 논/B 의/I
s/B k/I t/I 통/B 신/I 장/I 애/I 보/B 상/I ,/I "/B 약/I 관/I 이/B 상/I 보/B 상/I 하/I 겠/I 다/I "/I ./I ./I ./I 곧/I 세/B 부/I 보/B 상/I 발/B 표/I
잭/B 울/I 프/I 스/I 킨/I ,/I 독/B 일/I 체/B 험/I 이/B 벤/I 트/I '/B 진/I 짜/I 독/B 일/I 을/I 만/B 나/I 다/I '/I 진/B 행/I
'/B 사/I 회/I 적/I 경/I 제/I 배/B 우/I 자/I '/I 시/B 민/I 참/I 여/I 토/B 크/I 콘/B 서/I 트/I 개/B 최/I
</B 파/I 리/I 주/B 간/I 지/I 테/B 러/I >/I 세/B 계/I 각/B 국/I 풍/B 자/I 만/I 화/I 가/I 들/I ,/I 佛/B '/B 샤/I 를/I 리/I 엡/I 도/I '/I 비/B 극/I 만/B 화/I 로/I 애/B 도/I 물/B 결/I
밴/B 사/I 가/B 맹/I 점/I 카/B 드/I 결/I 제/I 1/B 6/I 9/I 억/I 불/B 법/I 리/B 베/I 이/I 트/I 적/B 발/I
방/B 통/I 위/I 또/B 다/I 시/I 보/B 조/I 금/I 조/B 사/I ./I ./I ./I 주/B 도/I 사/I 업/I 자/I '/B 엄/I 벌/I '/I
中/B -/I 사/I 우/I 디/I 석/B 유/I 공/I 룡/I 들/I 의/I '/B 밀/I 착/I '/I ./I ./I ./I 원/I 유/I 시/I 장/I 판/B 도/I 변/I 화/I 주/B 목/I
한/B 국/I 인/I 3/B 명/I 중/B 2/B 명/I 은/I "/B 일/I 본/I 혐/B 오/I "/I
c/B j/I 대/I 한/I 통/I 운/I '/B 이/I 라/I 크/I 재/B 건/I '/I 물/B 류/I 특/I 수/I


In [20]:
def corpus2sent(path):
    corpus = codecs.open(path, encoding='utf-8').read()
    raws = corpus.split('\n')
    sentences = []
    for raw in raws:
        tokens = raw.split(' ')
        sentence = []
        for token in tokens:
            try:
                word, tag = token.split('/')
                if word and tag:
                    sentence.append([word, tag])
            except:
                pass
        sentences.append(sentence)
    return sentences


In [21]:
sent0 = corpus2sent('train.txt')[0]

In [28]:
sent0

[['한', 'B'],
 ['미', 'I'],
 ['f', 'B'],
 ['t', 'I'],
 ['a', 'I'],
 ['이', 'B'],
 ['행', 'I'],
 ['위', 'I'],
 [',', 'I'],
 ['美', 'B'],
 ['관', 'B'],
 ['세', 'I'],
 ['폭', 'I'],
 ['탄', 'I'],
 ['근', 'B'],
 ['거', 'I'],
 ['규', 'B'],
 ['정', 'I'],
 ['논', 'B'],
 ['의', 'I']]

In [29]:
def index2feature(sent, i, offset):
    word, tag = sent[i + offset]
    if offset < 0:
        sign = ''
    else:
        sign = '+'
    return '{}{}:word={}'.format(sign, offset, word)

def word2features(sent, i):
    L = len(sent)
    word, tag = sent[i]
    features = ['bias']
    features.append(index2feature(sent, i, 0))
    if i > 1:
        features.append(index2feature(sent, i, -2))
    if i > 0:
        features.append(index2feature(sent, i, -1))
    else:
        features.append('bos')
    if i < L - 2:
        features.append(index2feature(sent, i, 2))
    if i < L - 1:
        features.append(index2feature(sent, i, 1))
    else:
        features.append('eos')
    return features

def sent2words(sent):
    return [word for word, tag in sent]

def sent2tags(sent):
    return [tag for word, tag in sent]

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [30]:
raw2corpus(os.path.join(INPUT_DIR, 'doc1.0e+4_typetitle.sentences'), 'test.txt')

In [31]:
train_sents = corpus2sent('train.txt')
test_sents = corpus2sent('test.txt')
train_x = [sent2features(sent) for sent in train_sents]
train_y = [sent2tags(sent) for sent in train_sents]
test_x = [sent2features(sent) for sent in test_sents]
test_y = [sent2tags(sent) for sent in test_sents]
trainer = pycrfsuite.Trainer()
for x, y in zip(train_x, train_y):  # 파이썬2에서 돌렸다
    trainer.append(x, y)
trainer.train('space.crfsuite') # 저장가능.

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 12802
Seconds required: 0.331

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 2147483647
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 151054.136854
Feature norm: 1.000000
Error norm: 93255.123849
Active features: 12088
Line search trials: 1
Line search step: 0.000009
Seconds required for this iteration: 0.104

***** Iteration #2 *****
Loss: 129409.034209
Feature norm: 0.755092
Error norm: 32501.671656
Active features: 12802
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.055

***** Iteration #3 *****
Loss: 124776.976287
Feature norm: 0.808414
Error norm: 18231.892265
Active features: 12802
Line search trials: 1
Line search step: 1.000000
Seconds re

In [32]:
tagger = pycrfsuite.Tagger()
tagger.open('space.crfsuite') # 로드해서 사용가능.

In [36]:
sent = test_x[0]
# print("Sentence: ", ' '.join(sent2words(sent)))  
# print("Correct:  ", ' '.join(sent2tags(sent)))  # B I I B I I I B I I I
# print("Predicted:", ' '.join(tagger.tag(sent2features(sent))))  # B I I I I I I I I B I

In [37]:
pred_y = [tagger.tag(x) for x in test_x]

def flush(path, X, Y):
    result = codecs.open(path, 'w', encoding='utf-8')
    for x, y in zip(X, Y):
        result.write(' '.join(['{}/{}'.format(feature[1].split('=')[1], tag) for feature, tag in zip(x, y)]))
        result.write('\n')
    result.close()

flush('pred.txt', test_x, pred_y)
corpus2raw('pred.txt', 'raw_pred.txt')

NameError: name 'corpus2raw' is not defined